#### Notice
Topography.py provides functions for loading required datasets, such as 

1. the MEaSUREs InSAR-Based Antarctica Ice Velocity Map (Ver 2),
https://nsidc.org/data/nsidc-0484/versions/2

2. Surface Mass Balance provided by RACMO2.3p2,
https://doi.org/10.5194/tc-12-1479-2018
https://www.projects.science.uu.nl/iceclimate/publications/data/2018/vwessem2018_tc/RACMO_Yearly/

3. MEaSUREs ITS_LIVE Antarctic Grounded Ice Sheet Elevation Change (Ver 1),
https://nsidc.org/data/nsidc-0782/versions/1

4. and BedMachine Antarctica Topography.
https://nsidc.org/data/nsidc-0756/versions/3

These functions are used for conveniently loading datasets for subglacial topography inversion purpose. However, they are not generally applicable to other datasets or the above datasets if they changed their format after 2025. This tutorial demonstrates how to load those dataset through functions in Topography.py.

# Tutorial 1: Loading and Visualizing Ice Stream Data

Let's first import libraries used in this jupyter lab. Notice that prerequired libraries in gstatsMCMC must be installed even if they are not imported in the jupyter notebook interface here

__Make sure the Topography.py is in the same folder as this jupyter notebook, otherwise you may need to modify the import__

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import Topography
import xarray as xr
import verde as vd

First, lets decides a region in Antarctic that we are interested in

__Q1: Where is your glacier? What is the boundary of this glacier in polar stereographic coordinate? Answer the question and then modify the code as needed__

Answer:  Macayeal: [-655484.559304, -869857.390077,]
Bindschadler : [-603229.008753, -772097.922049]

In [2]:
# Change the following coordinate based on your studying region
# make sure the coordinate ends with 250 or 750 to match up with pre-compiled radar data

# Macayeal: [-655484.559304, -869857.390077,]
# Bindschadler : [-603229.008753, -772097.922049]

xmin = -1070750
xmax = -488250
ymin = -996750
ymax = -440250

In [3]:
# Determine the resolution of the topography map

resolution = 500

In [4]:
xmin = -9870750
xmax = -408250
ymin = -1056750
ymax = -550250
xx, yy = np.meshgrid(np.arange(xmin, xmax, resolution), 
                     np.arange(ymin, ymax, resolution))

In [5]:
# This is the shape of your study region
xx.shape

(1013, 18925)

In [6]:
data = {'x': xx.flatten(),
        'y': yy.flatten()}
df = pd.DataFrame(data)
df

,x,y
0,-9870750,-1056750
1,-9870250,-1056750
2,-9869750,-1056750
3,-9869250,-1056750
4,-9868750,-1056750
...,...,...
19171020,-410750,-550750
19171021,-410250,-550750
19171022,-409750,-550750
19171023,-409250,-550750


### First, lets load the surface elevation of your Glacier

In [ ]:

bedmap_mask, bedmap_surf, bedmap_bed, bedmap_bed_uncertainty, fig = Topography.load_bedmap('/Volumes/tylerleSSD/DEMOGORGN_data/bedmap3.nc', 
                                                                                xx, yy, resolution)
fig

NOTICE! The region mask in bedmap will automatically be interpolated using nearest neighbor interpolation method


In [ ]:
df['surf'] = bedmap_surf.flatten()

### Next, lets load the pre-compiled ice thickness measured by ice-penetrating radar.

In [ ]:
xmin_ant = -3333250
xmax_ant = 3333250
ymin_ant = -3333250
ymax_ant = 3333250

xx_ant, yy_ant = np.meshgrid(np.arange(xmin_ant, xmax_ant + resolution, resolution), 
                     np.arange(ymin_ant, ymax_ant + resolution, resolution))

In [ ]:
xmin_ind = np.where(xx_ant == xmin)[1][0]
xmax_ind = np.where(xx_ant == xmax)[1][0]
ymin_ind = np.where(yy_ant == ymin)[0][0]
ymax_ind = np.where(yy_ant == ymax)[0][0]

In [ ]:
# replace with the directory of your downloaded data

file_path = '/Volumes/tylerleSSD/DEMOGORGN_data/bedmap3_raw_thick_compilation.npy'
radar_thickness = np.load(file_path)
radar_thickness = np.flip(radar_thickness,axis=0)
radar_thickness = radar_thickness[ymin_ind:ymax_ind,xmin_ind:xmax_ind]

In [ ]:
plt.pcolormesh(xx,yy,radar_thickness)
plt.colorbar()
plt.axis('scaled')

In [ ]:
df['bed'] = df['surf'].values - radar_thickness.flatten()

In [ ]:
plt.pcolormesh(xx,yy,df['bed'].values.reshape(xx.shape))
plt.colorbar()
plt.axis('scaled')

### Then we can load other data needed

Please refers to the document of the library to findout what to fill in each argument of the method. Feel free to contact your TA if you encountered any problem

In [ ]:
print('loading InSAR_MEaSUREs velocity dataset')
velx, vely, velxerr, velyerr, figvel = Topography.load_vel_measures('/Volumes/tylerleSSD/DEMOGORGN_data/antarctica_ice_velocity_450m_v2.nc', 
                                                                    xx, yy, resolution)
figvel

load_dhdt returns annual average of surface elevation change

__Q2: What time period did you choose for loading this dataset? Why? How the data change if you choose other time periods?__

Answer: 2014-2016. It has the readily available data for my ice streams. The flight paths changed for each year, so the ice velocity graph changed in terms of smoothing areas.

In [ ]:
print('loading height change rate dataset')
# change the time of the dataset
dhdt, figdhdt = Topography.load_dhdt('/Volumes/tylerleSSD/DEMOGORGN_data/ANT_G1920_GroundedIceHeight_v01.nc',xx,yy,resolution,
                                     interp_method='linear',begin_year=2014,end_year=2016,month=7)
figdhdt

# Ice flowing down Ice Rumples

__Notice! This function load for yearly surface mass balance. Does the year here match with the year you used for surface elevation?__

In [ ]:
print('loading racmo dataset')   
smb2014, fig1 = Topography.load_smb_racmo('/Volumes/tylerleSSD/DEMOGORGN_data/SMB_RACMO2.3p2_yearly_ANT27_1979_2016.nc', xx, yy, resolution, interp_method='linear',time=2014)
smb2015, fig2 = Topography.load_smb_racmo('/Volumes/tylerleSSD/DEMOGORGN_data/SMB_RACMO2.3p2_yearly_ANT27_1979_2016.nc', xx, yy, resolution, interp_method='linear',time=2015)
smb2016, fig3 = Topography.load_smb_racmo('/Volumes/tylerleSSD/DEMOGORGN_data/SMB_RACMO2.3p2_yearly_ANT27_1979_2016.nc', xx, yy, resolution, interp_method='linear',time=2016)
smb = (smb2014 + smb2015 + smb2016)/3
plt.pcolormesh(xx,yy,smb)

BedMachine use a different geoid compared to BedMap, so if you want to compare bed topography or surface elevation between BedMachine and Bedmap, geoid conversion need to be done. 

But if you are just comparing the thickness, there is no need for geoid conversion. We use this little trick here to obtain ice thickness information from BedMachine instead

In [ ]:
print('loading BedMachine dataset')
bm_mask, bm_source, bm_bed, bm_surface, bm_errbed, figbm = Topography.load_bedmachine('/Volumes/tylerleSSD/DEMOGORGN_data/BedMachineAntarctica-v3.nc', xx, yy, resolution)
figbm

In [ ]:
bm_thickness = bm_surface - bm_bed

__How will you interpret the data loaded?__
There are valleys or elevation change underneath the ice mask (ice bed).
The ice is elevated on the left side, and deescalate to the right. 
We can see that the flight path is sparse on the bottom. So more i

__Have you noticed any data that have gaps inside the region? Do you have any guesses why those gaps exists?__

These gaps could affect the calculation of ice flux residuals in the later tutorial. If the gaps exist, please try to load data from other datasets or consistently change the reference time, until you have found surface ice velocity, surface mass balance, and surface elevation change that covers the entire region

### Lets determine the high-velocity region, where we want to apply MCMC algorithm

In addition, since we have the velocity data, it will also be nice to know where the high velocity region is, remembering that the mass conservation technique is best applied in high velocity region

In [ ]:
# calculate velocity magnitude
vel_mag = np.sqrt(velx**2 + vely**2)

# Plot the velocity magnitude
plt.pcolormesh(xx,yy,vel_mag)
plt.colorbar()
plt.axis('scaled')
plt.title('velocity magnitude (m/yr)')
plt.xlabel('easting (m)')
plt.ylabel('northing (m)')
plt.show()

In [ ]:
# calculate velocity magnitude
vel_mag = np.sqrt(velx**2 + vely**2)

# Plot the velocity magnitude
plt.pcolormesh(xx,yy,vel_mag, vmax=50)
plt.colorbar()
plt.axis('scaled')
plt.title('velocity magnitude (m/yr) (cap at 50 m/yr)')
plt.xlabel('easting (m)')
plt.ylabel('northing (m)')
plt.show()

The *highvel_mask* returned try to smoothly enclose a region of high velocity and excluding locations where the ice is not grounded. This region could be used later to constrain the sampling location of the MCMC.

The function works as first find high velocity region, then smooth the boundary of this region. Because this smoothing generally will shrink the boundary, the boundary is later expanded outward for *distance_max* meters.

The degree of smoothness of the boundary could be modified by the optional argument *smooth_mode*, which is default to 10. A higher smooth_mode will give a smoother boundary

The *ocean_mask* will be 1 when the location is ocean (open ocean water without ice, sea ice, or ice shelf), and will be 0 otherwise.

The *grounded_ice_mask* is 1 at location where ice is present and grounded, and will be 0 otherwise.

An example where *ocean_mask == 0 and grounded_ice_mask == 0* will be ice-free terrestrial land

__Q3: Change distance_max and smooth_mode parameters, how do you see the high velocity region changed by these parameter? Which parameter give a reasonable answer?__

Answer: ____ 

Notice, we aims to cover region with 50 m/yr velocity magnitude

In [ ]:
# calculate high velocity region
ocean_mask = (bedmap_mask == 3) | (np.isnan(bedmap_mask)) # utilize the mask in BedMachine dataset to characterize ice regions
grounded_ice_mask = (bedmap_mask == 1) | (bedmap_mask == 2)
distance_max = 3500 # CHANGE
velocity_threshold = 100 
smooth_mode = 18 # CHANGE 
highvel_mask = Topography.get_highvel_boundary(velx, vely, velocity_threshold, grounded_ice_mask, ocean_mask, 
                                               distance_max, xx, yy, smooth_mode = smooth_mode)


In [ ]:
plt.pcolormesh(xx,yy,highvel_mask)
plt.axis('scaled')
plt.title('high velocity region')


### Reserved this space for sanity check. 
Have you visualize every data loaded? Do they look realistic? Has interpolating data give back artifacts? Is there any bed elevation measurement higher than elevation of ice surface?

### Save the loaded data

save the gridded radar data

In [ ]:
df['velx'] = velx.flatten()
df['vely'] = vely.flatten()
df['smb'] = ((smb2014 + smb2015 + smb2016)/3).flatten()
df['dhdt'] = dhdt.flatten()

df['highvel_mask'] = highvel_mask.flatten()


df['bedmap_mask'] = bedmap_mask.flatten()
df['bedmap_surf'] = bedmap_surf.flatten()
df['bedmachine_source'] = bm_source.flatten()
df['bedmachine_thickness'] = bm_thickness.flatten()
df['bedmap_bed'] = bedmap_bed.flatten()

In [ ]:
#Yay! Now just change the glacier name to your glacier
df.to_csv('BindSchalder_Macayeal_IceStreams.csv', index = False)

### Optional! Do not recommend to run in class

This is only for reference, in case if you are interested in how the radar data were compiled. Takes a long time and large storage space to run.

Please check with bedmap3 website and __download all the bed elevation measurements data into one folder__. Let's call that folder *radar_folder*.

In [26]:
# Change the radar_folder to the directory of the downloaded radar data
radar_folder = '/Volumes/tylerleSSD/DEMOGORGN_data/radarTest'
output_file_name = '/Volumes/tylerleSSD/DEMOGORGN_data/compiledRadar.csv'

Loading radar dataset

In [ ]:
df, df_out, fig = Topography.load_radar(radar_folder, output_file_name, include_only_thickness_data=False)

/Users/tienle/Glaciers/Topography.py:384: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(folder_path + '/' + filename,skiprows=18)
/Users/tienle/Glaciers/Topography.py:384: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(folder_path + '/' + filename,skiprows=18)
/Users/tienle/Glaciers/Topography.py:384: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(folder_path + '/' + filename,skiprows=18)
/Users/tienle/Glaciers/Topography.py:384: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(folder_path + '/' + filename,skiprows=18)
/Users/tienle/Glaciers/Topography.py:384: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(folder_path + '/' + filenam

the following files are loaded: 
['BAS_2015_FISS_AIR_BM3.csv', 'RNRF_2011_RAE_AIR_BM3.csv', 'BAS_2018_Thwaites_AIR_BM3.csv', 'NASA_2014_ICEBRIDGE_AIR_BM3.csv', 'INGV_1997_Talos-Dome_AIR_BM3.csv', 'LDEO_2015_ROSETTA_AIR_BM3.csv', 'AWI_2018_JURAS_AIR_BM3.csv', 'RNRF_1975_Lazarev_SEI_BM3.csv', 'INGV_2003_Talos-Dome_AIR_BM3.csv', 'CRESIS_2013_Siple-Coast_AIR_BM3.csv', 'CRESIS_2009_AntarcticaTO_AIR_BM3.csv', 'UTIG_2013_GIMBLE_AIR_BM3.csv', 'UTIG_2015_EAGLE_AIR_BM3.csv', 'RNRF_2019_RAE_AIR_BM3.csv', 'PRIC_2017_CHA3_AIR_BM3.csv', 'RNRF_2016_RAE_AIR_BM3.csv', 'UWASHINGTON_2018_South-Pole-Lake_GRN_BM3.csv', 'AWI_2015_GEA-DML_AIR_BM3.csv', 'INGV_1999_Talos-Dome_AIR_BM3.csv', 'RNRF_2003_AMSap5_AIR_BM3.csv', 'AWI_2019_JURAS_AIR_BM3.csv', 'BAS_2011_Adelaide_AIR_BM3.csv', 'NASA_2016_ICEBRIDGE_AIR_BM3.csv', 'RNRF_2008_AMSap5_AIR_BM3.csv', 'RNRF_2005_AMSap5_AIR_BM3.csv', 'ULB_2012_ICECON_GRN_BM3.csv', 'BAS_2008_Lake-Ellsworth_GRN_BM3.csv', 'BAS_2019_Thwaites_AIR_BM3.csv', 'RNRF_2018_RAE_AIR_BM3.csv', 

In [ ]:
df

In [ ]:
fig

gridding radar data

In [ ]:
df=df[(df['x']>=xmin) & (df['x']<=xmax) & (df['y']>=ymin) & (df['y']<=ymax)] # find all radar data inside the domain
df_grid, grid_matrix, rows, cols = Topography.grid_data(df, 'x', 'y', 'bed', resolution, xmin, xmax, ymin, ymax) # grid them
df_grid = df_grid.rename(columns = {"Z": "bed"})

x_uniq = np.unique(df_grid.X)
y_uniq = np.unique(df_grid.Y)

xx, yy = np.meshgrid(x_uniq, y_uniq) # xx store the x coordinate of every location on this 2D map. Similarly, yy store y coordinate
xx.shape

In [ ]:
df_grid

plt.pcolormesh(xx,yy,df_grid['bed'].values.reshape(xx.shape))
plt.colorbar()
plt.axis('scaled')
plt.title('gridded radar measurements')